# Ugesco : enrich and desambiguate a sample of the Samnang-Hans' JSON

In [51]:
from ugesco import *

In [52]:
#test sur un Json contenant 20 éléments pris au hasard
with open(r"C:\Users\ettor\Documents\GitHub\UGESCO\data\jsondata_ugesco.json", "r", encoding="utf8") as file:
    json = json.load(file)
    
sample = random.sample(json, k=100)


In [53]:
data = pd.DataFrame([flatten(line) for line in sample])

In [54]:
data.head()

,beeldid,imageclassification_class1,imageclassification_class2,imageclassification_class3,imageclassification_class4,imageclassification_class5,imageclassification_prob1,imageclassification_prob2,imageclassification_prob3,imageclassification_prob4,...,spatial_value_3,spatial_value_4,spatial_value_5,temporal,temporal_key_0,temporal_key_1,temporal_key_2,temporal_value_0,temporal_value_1,temporal_value_2
0,00029158,plaza,courthouse,courtyard,schoolhouse,campus,0.419811,0.181694,0.051251,0.033715,...,NaN,NaN,NaN,NaN,TIMEX_DATE,NaN,NaN,4 septembre 1944,NaN,NaN
1,00037482,street,plaza,medina,bazaar/outdoor,alley,0.833741,0.101289,0.041964,0.007429,...,NaN,NaN,NaN,NaN,TIMEX_TIME,TIMEX_DATE,NaN,ce matin,13/7/1936,NaN
2,00040192,bazaar/outdoor,pavilion,plaza,courtyard,beer_garden,0.175289,0.167766,0.139539,0.107811,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00013898,train_station/platform,subway_station/platform,hangar/outdoor,railroad_track,bus_station/indoor,0.974537,0.017484,0.003460,0.001479,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00008022,plaza,slum,campus,office_building,street,0.104124,0.070034,0.067388,0.052687,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
# exclude image classifications and temporal values for the moment
data = data[data.columns.drop(list(data.filter(regex='imageclassification|temporal')))]

In [56]:
# reshape the dataframe verticaly
data = data.set_index('beeldid')
data.columns = data.columns.str.split('_', expand=True)

In [57]:
data = data.stack().reset_index(level=1, drop=True).reset_index()

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2910: PerformanceWarning: indexing past lexsort depth may impact performance.
  exec(code_obj, self.user_global_ns, self.user_ns)
D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2910: PerformanceWarning: indexing past lexsort depth may impact performance.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [58]:
data.head()

beeldid                           spatial                \
                                        value           key   
0  00029158                         Bruxelles     LOC_PLACE   
1  00037482  la Fraternelle de la Gendarmerie  LOC_BUILDING   
2  00037482                         Bruxelles     LOC_PLACE   
3  00040192                            Anvers     LOC_PLACE   
4  00013898                Gare de Schaerbeek  LOC_BUILDING   

                                     
                              value  
0                         Bruxelles  
1  la Fraternelle de la Gendarmerie  
2                         Bruxelles  
3                            Anvers  
4                Gare de Schaerbeek

In [59]:
data.columns = ['_'.join(col) for col in data.columns]

In [60]:
data.rename(columns={'beeldid_': 'beeldid'}, inplace=True)

In [61]:
data.head()

,beeldid,spatial_value,spatial_key,spatial_value
0,00029158,Bruxelles,LOC_PLACE,Bruxelles
1,00037482,la Fraternelle de la Gendarmerie,LOC_BUILDING,la Fraternelle de la Gendarmerie
2,00037482,Bruxelles,LOC_PLACE,Bruxelles
3,00040192,Anvers,LOC_PLACE,Anvers
4,00013898,Gare de Schaerbeek,LOC_BUILDING,Gare de Schaerbeek


In [62]:
#join/merge with phototheque_pallas to get the locations. 
#The csv is gzipped to get around the 100MB file weigth limitation of Github
phototeque = pd.read_csv("data/phototheque_pallas.csv.gz", compression="gzip", encoding="utf8", dtype={'beeldid': str, 'trfwnumm': str})

data = pd.merge(data, phototeque, how='left', on=['beeldid'], suffixes=['', '_x'])

In [63]:
# Pictures that contains more than a loc in the thesaurus descriptors

#data[data['loc_qid'].str.contains(",", na = False)]

In [64]:
data.head()

,beeldid,spatial_value,spatial_key,spatial_value,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name
0,00029158,Bruxelles,LOC_PLACE,Bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,4 septembre 1944 (Photo René Caucheteux - Brux...,"Libération de Bruxelles par les allies., 9/194...",1625,bruxelles (belgique)--histoire--1944 (liberation),Q31,BE,Belgique,Q239,Bruxelles
1,00037482,la Fraternelle de la Gendarmerie,LOC_BUILDING,la Fraternelle de la Gendarmerie,http://pallas.cegesoma.be/pls/opac/opac.search...,Le roi Leopold remet un drapeau à la fraternel...,"Le roi Léopold III, [...-1939]. - 573 photos",3140,NaN,NaN,NaN,NaN,NaN,NaN
2,00037482,Bruxelles,LOC_PLACE,Bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,Le roi Leopold remet un drapeau à la fraternel...,"Le roi Léopold III, [...-1939]. - 573 photos",3140,NaN,NaN,NaN,NaN,NaN,NaN
3,00040192,Anvers,LOC_PLACE,Anvers,http://pallas.cegesoma.be/pls/opac/opac.search...,Vous n'avez rien à déclarer ? L'arrivée d'un p...,Navigation : [...-1939]. - 63 photos,3276,navigation--belgique,Q31,BE,Belgique,Q239,Bruxelles
4,00040192,Anvers,LOC_PLACE,Anvers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q3100236,Lanaye


In [65]:
# Merge with NER_classes (spatial_keys matched with wikidata)
ner_classes = pd.read_csv("data/ner_classes.csv")

data = pd.merge(data, ner_classes, how='left', left_on = ['spatial_key'], right_on = ['ner_class'])

In [66]:
data.head()

,beeldid,spatial_value,spatial_key,spatial_value,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name,ner_class,wiki_qid,wiki_class
0,00029158,Bruxelles,LOC_PLACE,Bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,4 septembre 1944 (Photo René Caucheteux - Brux...,"Libération de Bruxelles par les allies., 9/194...",1625,bruxelles (belgique)--histoire--1944 (liberation),Q31,BE,Belgique,Q239,Bruxelles,LOC_PLACE,Q56061,administrative territorial entity
1,00037482,la Fraternelle de la Gendarmerie,LOC_BUILDING,la Fraternelle de la Gendarmerie,http://pallas.cegesoma.be/pls/opac/opac.search...,Le roi Leopold remet un drapeau à la fraternel...,"Le roi Léopold III, [...-1939]. - 573 photos",3140,NaN,NaN,NaN,NaN,NaN,NaN,LOC_BUILDING,Q41176,building
2,00037482,Bruxelles,LOC_PLACE,Bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,Le roi Leopold remet un drapeau à la fraternel...,"Le roi Léopold III, [...-1939]. - 573 photos",3140,NaN,NaN,NaN,NaN,NaN,NaN,LOC_PLACE,Q56061,administrative territorial entity
3,00040192,Anvers,LOC_PLACE,Anvers,http://pallas.cegesoma.be/pls/opac/opac.search...,Vous n'avez rien à déclarer ? L'arrivée d'un p...,Navigation : [...-1939]. - 63 photos,3276,navigation--belgique,Q31,BE,Belgique,Q239,Bruxelles,LOC_PLACE,Q56061,administrative territorial entity
4,00040192,Anvers,LOC_PLACE,Anvers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q3100236,Lanaye,LOC_PLACE,Q56061,administrative territorial entity


In [67]:
# rename the new columns merged
data.rename(columns={'wiki_qid': 'ner_class_qid', 'wiki_class':'ner_class_name'}, inplace=True)

In [68]:
#remove duplicate columns
data = data.T.drop_duplicates().T

In [69]:
data.head()

,beeldid,spatial_value,spatial_key,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name,ner_class_qid,ner_class_name
0,00029158,Bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,4 septembre 1944 (Photo René Caucheteux - Brux...,"Libération de Bruxelles par les allies., 9/194...",1625,bruxelles (belgique)--histoire--1944 (liberation),Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
1,00037482,la Fraternelle de la Gendarmerie,LOC_BUILDING,http://pallas.cegesoma.be/pls/opac/opac.search...,Le roi Leopold remet un drapeau à la fraternel...,"Le roi Léopold III, [...-1939]. - 573 photos",3140,NaN,NaN,NaN,NaN,NaN,NaN,Q41176,building
2,00037482,Bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Le roi Leopold remet un drapeau à la fraternel...,"Le roi Léopold III, [...-1939]. - 573 photos",3140,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
3,00040192,Anvers,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Vous n'avez rien à déclarer ? L'arrivée d'un p...,Navigation : [...-1939]. - 63 photos,3276,navigation--belgique,Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
4,00040192,Anvers,LOC_PLACE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q3100236,Lanaye,Q56061,administrative territorial entity


In [70]:
# remove articles in spatial_value
data['spatial_value'] = data['spatial_value'].str.replace(pat=r"\b(le|la|l\s+'|l'|les)\b", repl="", n=1, case=False)

In [71]:
#data.to_csv(r"C:\Users\ettor\Desktop\data_ugesco.csv", header=True, index=False, encoding="utf-8")

In [72]:
data.head()

,beeldid,spatial_value,spatial_key,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name,ner_class_qid,ner_class_name
0,00029158,Bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,4 septembre 1944 (Photo René Caucheteux - Brux...,"Libération de Bruxelles par les allies., 9/194...",1625,bruxelles (belgique)--histoire--1944 (liberation),Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
1,00037482,Fraternelle de la Gendarmerie,LOC_BUILDING,http://pallas.cegesoma.be/pls/opac/opac.search...,Le roi Leopold remet un drapeau à la fraternel...,"Le roi Léopold III, [...-1939]. - 573 photos",3140,NaN,NaN,NaN,NaN,NaN,NaN,Q41176,building
2,00037482,Bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Le roi Leopold remet un drapeau à la fraternel...,"Le roi Léopold III, [...-1939]. - 573 photos",3140,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
3,00040192,Anvers,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Vous n'avez rien à déclarer ? L'arrivée d'un p...,Navigation : [...-1939]. - 63 photos,3276,navigation--belgique,Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
4,00040192,Anvers,LOC_PLACE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q3100236,Lanaye,Q56061,administrative territorial entity


In [73]:
#normalize spatial_value
data['spatial_value'] = data['spatial_value'].apply(lambda x: Fingerprinter(x).get_fingerprint_nonsorted())

In [74]:
#transformer les loc_place pays en loc_country
with open(r"C:\Users\ettor\Documents\GitHub\UGESCO\data\countries.csv", 'r', encoding="utf8") as f:
    countries = set([Fingerprinter(country).get_fingerprint_nonsorted() for country in f])

# Grab DataFrame rows where column value is present in a list
data.loc[(data['spatial_value'].isin(countries)), ['spatial_key']] = "LOC_COUNTRY"

In [75]:
data

,beeldid,spatial_value,spatial_key,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name,ner_class_qid,ner_class_name
0,00029158,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,4 septembre 1944 (Photo René Caucheteux - Brux...,"Libération de Bruxelles par les allies., 9/194...",1625,bruxelles (belgique)--histoire--1944 (liberation),Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
1,00037482,fraternelle de la gendarmerie,LOC_BUILDING,http://pallas.cegesoma.be/pls/opac/opac.search...,Le roi Leopold remet un drapeau à la fraternel...,"Le roi Léopold III, [...-1939]. - 573 photos",3140,NaN,NaN,NaN,NaN,NaN,NaN,Q41176,building
2,00037482,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Le roi Leopold remet un drapeau à la fraternel...,"Le roi Léopold III, [...-1939]. - 573 photos",3140,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
3,00040192,anvers,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Vous n'avez rien à déclarer ? L'arrivée d'un p...,Navigation : [...-1939]. - 63 photos,3276,navigation--belgique,Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
4,00040192,anvers,LOC_PLACE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q3100236,Lanaye,Q56061,administrative territorial entity
5,00040192,anvers,LOC_PLACE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q267663,Hoboken,Q56061,administrative territorial entity
6,00040192,anvers,LOC_PLACE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q12892,Anvers,Q56061,administrative territorial entity
7,00040192,anvers,LOC_PLACE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q12996,Ostende,Q56061,administrative territorial entity
8,00013898,gare de schaerbeek,LOC_BUILDING,http://pallas.cegesoma.be/pls/opac/opac.search...,Gare de Schaerbeek. Départ du train ramenant e...,Départ d'enfants de Schaerbeek pour les coloni...,966,NaN,Q31,BE,Belgique,Q12887,Schaerbeek,Q41176,building
9,00013898,espagne,LOC_COUNTRY,http://pallas.cegesoma.be/pls/opac/opac.search...,Gare de Schaerbeek. Départ du train ramenant e...,Départ d'enfants de Schaerbeek pour les coloni...,966,NaN,Q31,BE,Belgique,Q12887,Schaerbeek,Q56061,administrative territorial entity


In [76]:
gb = data.groupby(['beeldid'])

In [77]:
gb.head() 

,beeldid,spatial_value,spatial_key,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name,ner_class_qid,ner_class_name
0,00029158,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,4 septembre 1944 (Photo René Caucheteux - Brux...,"Libération de Bruxelles par les allies., 9/194...",1625,bruxelles (belgique)--histoire--1944 (liberation),Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
1,00037482,fraternelle de la gendarmerie,LOC_BUILDING,http://pallas.cegesoma.be/pls/opac/opac.search...,Le roi Leopold remet un drapeau à la fraternel...,"Le roi Léopold III, [...-1939]. - 573 photos",3140,NaN,NaN,NaN,NaN,NaN,NaN,Q41176,building
2,00037482,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Le roi Leopold remet un drapeau à la fraternel...,"Le roi Léopold III, [...-1939]. - 573 photos",3140,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
3,00040192,anvers,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Vous n'avez rien à déclarer ? L'arrivée d'un p...,Navigation : [...-1939]. - 63 photos,3276,navigation--belgique,Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
4,00040192,anvers,LOC_PLACE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q3100236,Lanaye,Q56061,administrative territorial entity
5,00040192,anvers,LOC_PLACE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q267663,Hoboken,Q56061,administrative territorial entity
6,00040192,anvers,LOC_PLACE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q12892,Anvers,Q56061,administrative territorial entity
7,00040192,anvers,LOC_PLACE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q12996,Ostende,Q56061,administrative territorial entity
8,00013898,gare de schaerbeek,LOC_BUILDING,http://pallas.cegesoma.be/pls/opac/opac.search...,Gare de Schaerbeek. Départ du train ramenant e...,Départ d'enfants de Schaerbeek pour les coloni...,966,NaN,Q31,BE,Belgique,Q12887,Schaerbeek,Q41176,building
9,00013898,espagne,LOC_COUNTRY,http://pallas.cegesoma.be/pls/opac/opac.search...,Gare de Schaerbeek. Départ du train ramenant e...,Départ d'enfants de Schaerbeek pour les coloni...,966,NaN,Q31,BE,Belgique,Q12887,Schaerbeek,Q56061,administrative territorial entity


In [78]:
#Sélectionner les records qui contiennent moins de 3 loc_place (attention : brussels et bruxelles = 1 en fait. D'où, deux semble un bon compromis)
df_test = gb['spatial_key'].apply(lambda x: (x=='LOC_PLACE').sum()).reset_index(name='count_places')

In [79]:
df_test

,beeldid,count_places
0,00000047,1
1,00002203,1
2,00002895,0
3,00004361,0
4,00005191,1
5,00005314,0
6,00005400,0
7,00005510,1
8,00005541,1
9,00005548,0


In [80]:
filtre = list(df_test[df_test['count_places'] < 3]['beeldid'])  
gb_filtered = data[data['beeldid'].isin(filtre)].groupby('beeldid')

In [81]:
gb_filtered.head()

,beeldid,spatial_value,spatial_key,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name,ner_class_qid,ner_class_name
0,00029158,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,4 septembre 1944 (Photo René Caucheteux - Brux...,"Libération de Bruxelles par les allies., 9/194...",1625,bruxelles (belgique)--histoire--1944 (liberation),Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
1,00037482,fraternelle de la gendarmerie,LOC_BUILDING,http://pallas.cegesoma.be/pls/opac/opac.search...,Le roi Leopold remet un drapeau à la fraternel...,"Le roi Léopold III, [...-1939]. - 573 photos",3140,NaN,NaN,NaN,NaN,NaN,NaN,Q41176,building
2,00037482,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Le roi Leopold remet un drapeau à la fraternel...,"Le roi Léopold III, [...-1939]. - 573 photos",3140,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
8,00013898,gare de schaerbeek,LOC_BUILDING,http://pallas.cegesoma.be/pls/opac/opac.search...,Gare de Schaerbeek. Départ du train ramenant e...,Départ d'enfants de Schaerbeek pour les coloni...,966,NaN,Q31,BE,Belgique,Q12887,Schaerbeek,Q41176,building
9,00013898,espagne,LOC_COUNTRY,http://pallas.cegesoma.be/pls/opac/opac.search...,Gare de Schaerbeek. Départ du train ramenant e...,Départ d'enfants de Schaerbeek pour les coloni...,966,NaN,Q31,BE,Belgique,Q12887,Schaerbeek,Q56061,administrative territorial entity
10,00008022,place du jeu de balle,LOC_SPACES,http://pallas.cegesoma.be/pls/opac/opac.search...,Des baraquements pour le Secours d'Hiver. Le S...,"Diverses actions sociales en Belgique, 1940-19...",279,aide sociale--belgique,Q31,BE,Belgique,NaN,NaN,Q294440,public space
11,00026722,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Bombardement terroriste de Bruxelles le 7/9 pa...,"Bombardement de Bruxelles par les alliés, 7/9/...",1589,bruxelles (belgique)--histoire--1943 (bombarde...,Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
12,00039705,pavillon,LOC_BUILDING,http://pallas.cegesoma.be/pls/opac/opac.search...,[Grand Duché de Luxembourg]. Sortie du Prince ...,"Exposition universelle à Bruxelles, 1935. - 11...",3218,NaN,NaN,NaN,NaN,NaN,NaN,Q41176,building
13,00039783,pavillon ' italie,LOC_BUILDING,http://pallas.cegesoma.be/pls/opac/opac.search...,Vue nocturne à l'Exposition de Bruxelles. Une ...,"Exposition universelle à Bruxelles, 1935. - 11...",3218,NaN,NaN,NaN,NaN,NaN,NaN,Q41176,building
14,00029146,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Troupes anglaises à Bruxelles sept 44,"Libération de Bruxelles par les allies., 9/194...",1625,bruxelles (belgique)--histoire--1944 (liberation),Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity


In [82]:
#et maintenant appliquer la concatenation uniquement si le record ne contient pas plus de deux loc_place
gb = pd.DataFrame(gb_filtered['spatial_value'].apply(lambda x: "%s" % ' '.join(x))).reset_index()

In [83]:
gb.head()

,beeldid,spatial_value
0,00000047,bruxelles rue de braie
1,00002203,flandre eglise
2,00002895,gare du nord
3,00004361,sablon
4,00005191,berlin


In [84]:
#comment concaténer loc_name maintenant, et accéder à country_name ?
#en attendant de trouver une solution élégante, récréer un dataset contenant ces infos

gb = gb.merge(data[["beeldid", "loc_name", "country_name"]])

In [85]:
gb.head()

,beeldid,spatial_value,loc_name,country_name
0,00000047,bruxelles rue de braie,Bruxelles,Belgique
1,00000047,bruxelles rue de braie,Bruxelles,Belgique
2,00002203,flandre eglise,NaN,Belgique
3,00002203,flandre eglise,NaN,Belgique
4,00002895,gare du nord,NaN,NaN


In [86]:
#on splitte les noms de pays multiples (n'existent plus)
#gb = tidy_split(gb.merge(data[["beeldid", "country_name", "loc_name"]]), column="country_name", sep=",", keep=False)

In [87]:
#on enlève les doublons
gb.drop_duplicates(subset=None, keep='first', inplace=True)

In [88]:
gb

,beeldid,spatial_value,loc_name,country_name
0,00000047,bruxelles rue de braie,Bruxelles,Belgique
2,00002203,flandre eglise,NaN,Belgique
4,00002895,gare du nord,NaN,NaN
5,00004361,sablon,NaN,Belgique
6,00005191,berlin,NaN,Allemagne
7,00005314,cimetieres de bruxelles,Bruxelles,Belgique
8,00005400,bois de cambre,Bruxelles,Belgique
9,00005510,place rogier bruxelles boulevards adolphe max ...,Bruxelles,Belgique
12,00005541,grand-place bruxelles europe,Bruxelles,Belgique
15,00005548,au petit sablon ' eglise du sablon statue des ...,Bruxelles,Belgique


In [89]:
#on normalise les lieux et pays
gb['loc_name'] = gb['loc_name'].apply(lambda x: Fingerprinter(str(x)).get_fingerprint_nonsorted()).str.replace('nan',  '')
gb['country_name'] = gb['country_name'].apply(lambda x: Fingerprinter(str(x)).get_fingerprint_nonsorted()).str.replace('nan',  '')

In [90]:
# on concatène le tout en supprimant les doublons

gb['loc_concat'] = gb['spatial_value'].fillna('').map(
    str) + ' ' + gb['loc_name'].fillna('').map(str) + ' ' + gb['country_name'].fillna('').map(str)

gb['loc_concat'] = gb['loc_concat'].apply(
    lambda x: ' '.join(pd.unique(x.split())))

In [91]:
gb

,beeldid,spatial_value,loc_name,country_name,loc_concat
0,00000047,bruxelles rue de braie,bruxelles,belgique,bruxelles rue de braie belgique
2,00002203,flandre eglise,,belgique,flandre eglise belgique
4,00002895,gare du nord,,,gare du nord
5,00004361,sablon,,belgique,sablon belgique
6,00005191,berlin,,allemagne,berlin allemagne
7,00005314,cimetieres de bruxelles,bruxelles,belgique,cimetieres de bruxelles belgique
8,00005400,bois de cambre,bruxelles,belgique,bois de cambre bruxelles belgique
9,00005510,place rogier bruxelles boulevards adolphe max ...,bruxelles,belgique,place rogier bruxelles boulevards adolphe max ...
12,00005541,grand-place bruxelles europe,bruxelles,belgique,grand-place bruxelles europe belgique
15,00005548,au petit sablon ' eglise du sablon statue des ...,bruxelles,belgique,au petit sablon ' eglise du statue des comtes ...


In [92]:
#appliquer get_nominatim. ATTENTION : ajouter le pays pour plus de précision

gb['nominatim'] = gb['loc_concat'].apply(get_nominatim)

D:\Anaconda3\lib\socket.py:657: ResourceWarning: unclosed <socket.socket fd=3304, family=AddressFamily.AF_INET6, type=SocketKind.SOCK_STREAM, proto=0, laddr=('2a02:1811:3700:1500:886d:dfde:a57f:2129', 53615, 0, 0), raddr=('2001:630:12:500:ec4:7aff:fe66:96d2', 443, 0, 0)>
  self._sock = None
D:\Anaconda3\lib\socket.py:657: ResourceWarning: unclosed <socket.socket fd=3364, family=AddressFamily.AF_INET6, type=SocketKind.SOCK_STREAM, proto=0, laddr=('2a02:1811:3700:1500:886d:dfde:a57f:2129', 53617, 0, 0), raddr=('2001:630:12:500:ec4:7aff:fe66:96d2', 443, 0, 0)>
  self._sock = None
D:\Anaconda3\lib\socket.py:657: ResourceWarning: unclosed <socket.socket fd=3268, family=AddressFamily.AF_INET6, type=SocketKind.SOCK_STREAM, proto=0, laddr=('2a02:1811:3700:1500:886d:dfde:a57f:2129', 53619, 0, 0), raddr=('2001:630:12:500:ec4:7aff:fe66:96d2', 443, 0, 0)>
  self._sock = None
D:\Anaconda3\lib\socket.py:657: ResourceWarning: unclosed <socket.socket fd=3320, family=AddressFamily.AF_INET6, type=Socket

D:\Anaconda3\lib\socket.py:657: ResourceWarning: unclosed <socket.socket fd=4084, family=AddressFamily.AF_INET6, type=SocketKind.SOCK_STREAM, proto=0, laddr=('2a02:1811:3700:1500:886d:dfde:a57f:2129', 53684, 0, 0), raddr=('2001:630:12:500:ec4:7aff:fe66:96d2', 443, 0, 0)>
  self._sock = None
D:\Anaconda3\lib\socket.py:657: ResourceWarning: unclosed <socket.socket fd=3388, family=AddressFamily.AF_INET6, type=SocketKind.SOCK_STREAM, proto=0, laddr=('2a02:1811:3700:1500:886d:dfde:a57f:2129', 53686, 0, 0), raddr=('2001:630:12:500:ec4:7aff:fe66:96d2', 443, 0, 0)>
  self._sock = None
D:\Anaconda3\lib\socket.py:657: ResourceWarning: unclosed <socket.socket fd=3124, family=AddressFamily.AF_INET6, type=SocketKind.SOCK_STREAM, proto=0, laddr=('2a02:1811:3700:1500:886d:dfde:a57f:2129', 53688, 0, 0), raddr=('2001:630:12:500:ec4:7aff:fe66:96d2', 443, 0, 0)>
  self._sock = None
D:\Anaconda3\lib\socket.py:657: ResourceWarning: unclosed <socket.socket fd=3144, family=AddressFamily.AF_INET6, type=Socket

In [93]:
gb.head(10)

,beeldid,spatial_value,loc_name,country_name,loc_concat,nominatim
0,00000047,bruxelles rue de braie,bruxelles,belgique,bruxelles rue de braie belgique,[]
2,00002203,flandre eglise,,belgique,flandre eglise belgique,"[{'place_id': '813951', 'licence': 'Data © Ope..."
4,00002895,gare du nord,,,gare du nord,"[{'place_id': '220124974', 'licence': 'Data © ..."
5,00004361,sablon,,belgique,sablon belgique,"[{'place_id': '4412425', 'licence': 'Data © Op..."
6,00005191,berlin,,allemagne,berlin allemagne,"[{'place_id': '226584215', 'licence': 'Data © ..."
7,00005314,cimetieres de bruxelles,bruxelles,belgique,cimetieres de bruxelles belgique,"[{'place_id': '7103870', 'licence': 'Data © Op..."
8,00005400,bois de cambre,bruxelles,belgique,bois de cambre bruxelles belgique,"[{'place_id': '179982083', 'licence': 'Data © ..."
9,00005510,place rogier bruxelles boulevards adolphe max ...,bruxelles,belgique,place rogier bruxelles boulevards adolphe max ...,[]
12,00005541,grand-place bruxelles europe,bruxelles,belgique,grand-place bruxelles europe belgique,[]
15,00005548,au petit sablon ' eglise du sablon statue des ...,bruxelles,belgique,au petit sablon ' eglise du statue des comtes ...,[]


In [94]:
#Comment attribuer un score de fiabilité aux résultats et choisir le meilleur s'il y en a un ?

def get_element(array, *args):
    """parse the first nominatim json."""
    for el in array:
        return dig(el, *args)
    
gb['nominatim_main_loc'] = np.vectorize(get_element)(gb['nominatim'], "display_name")
gb['nominatim_type_loc'] = np.vectorize(get_element)(gb['nominatim'], "type")
gb['nominatim_city_loc'] = np.vectorize(get_element)(gb['nominatim'], "address", "city")
gb['nominatim_country_loc'] = np.vectorize(get_element)(gb['nominatim'], "address", "country") 

In [95]:
gb 

,beeldid,spatial_value,loc_name,country_name,loc_concat,nominatim,nominatim_main_loc,nominatim_type_loc,nominatim_city_loc,nominatim_country_loc
0,00000047,bruxelles rue de braie,bruxelles,belgique,bruxelles rue de braie belgique,[],None,None,None,None
2,00002203,flandre eglise,,belgique,flandre eglise belgique,"[{'place_id': '813951', 'licence': 'Data © Ope...","Sint-Vincentius a Paulokerk, 189, Luipegem, Bo...",place_of_worship,None,Belgique
4,00002895,gare du nord,,,gare du nord,"[{'place_id': '220124974', 'licence': 'Data © ...","Gare du Nord (RER), Rue du Faubourg Saint-Deni...",station,Paris,France
5,00004361,sablon,,belgique,sablon belgique,"[{'place_id': '4412425', 'licence': 'Data © Op...","Sablon, Rue de la Régence, Quartier du Sablon,...",bicycle_rental,Ville de Bruxelles,Belgique
6,00005191,berlin,,allemagne,berlin allemagne,"[{'place_id': '226584215', 'licence': 'Data © ...","Berlin, 10117, Allemagne",city,Berlin,Allemagne
7,00005314,cimetieres de bruxelles,bruxelles,belgique,cimetieres de bruxelles belgique,"[{'place_id': '7103870', 'licence': 'Data © Op...","Cimetières, Drève des Maricolles, Berchem-Sain...",bus_stop,None,Belgique
8,00005400,bois de cambre,bruxelles,belgique,bois de cambre bruxelles belgique,"[{'place_id': '179982083', 'licence': 'Data © ...","Bois de la Cambre, Avenue de Diane, Bruxelles,...",wood,None,Belgique
9,00005510,place rogier bruxelles boulevards adolphe max ...,bruxelles,belgique,place rogier bruxelles boulevards adolphe max ...,[],None,None,None,None
12,00005541,grand-place bruxelles europe,bruxelles,belgique,grand-place bruxelles europe belgique,[],None,None,None,None
15,00005548,au petit sablon ' eglise du sablon statue des ...,bruxelles,belgique,au petit sablon ' eglise du statue des comtes ...,[],None,None,None,None


In [114]:
#apply wikipedia only if nominatim failed
gb['wikipedia_improved'] = gb[pd.isnull(gb.nominatim_main_loc)]['loc_concat'].apply(lambda x: gsrsearch(x))

In [115]:
#sélectionner les lignes où Nominatim et Wikipedia ont échoué
gb[(pd.isnull(gb.nominatim_main_loc)) & (pd.isnull(gb.wikipedia_improved))]

,beeldid,spatial_value,loc_name,country_name,loc_concat,nominatim,nominatim_main_loc,nominatim_type_loc,nominatim_city_loc,nominatim_country_loc,wikipedia_improved,wikidata
20,00005572,bruxelles boulevard anspacht galeries anspach,bruxelles,belgique,bruxelles boulevard anspacht galeries anspach ...,[],None,None,None,None,None,NaN
89,00030394,petite espinette chateau fond'roy,,,petite espinette chateau fond'roy,[],None,None,None,None,None,NaN
91,00032087,chateau fond'roy petite espinette,,,chateau fond'roy petite espinette,[],None,None,None,None,None,NaN
108,00036469,' entraide ' ixelles bruxelles ixelles nid san...,,belgique,' entraide ixelles bruxelles nid sans souci be...,[],None,None,None,None,None,NaN


In [123]:
#Chercher dans Wikidata les spatial_values individuelles là où tout a échoué. Pas oublier d'ungrouper.
gb['wikidata'] = gb[(pd.isnull(gb.nominatim_main_loc)) & (pd.isnull(gb.wikipedia_improved))]['spatial_value'].apply(lambda x: get_wikidata(x, 'Q618123'))

In [124]:
gb[(pd.isnull(gb.nominatim_main_loc)) & (pd.isnull(gb.wikipedia_improved))].reset_index(drop=True )

,beeldid,spatial_value,loc_name,country_name,loc_concat,nominatim,nominatim_main_loc,nominatim_type_loc,nominatim_city_loc,nominatim_country_loc,wikipedia_improved,wikidata
0,00005572,bruxelles boulevard anspacht galeries anspach,bruxelles,belgique,bruxelles boulevard anspacht galeries anspach ...,[],None,None,None,None,None,No match
1,00030394,petite espinette chateau fond'roy,,,petite espinette chateau fond'roy,[],None,None,None,None,None,No match
2,00032087,chateau fond'roy petite espinette,,,chateau fond'roy petite espinette,[],None,None,None,None,None,No match
3,00036469,' entraide ' ixelles bruxelles ixelles nid san...,,belgique,' entraide ixelles bruxelles nid sans souci be...,[],None,None,None,None,None,No match
